# Домашно №2 (Sliding Blocks)
Играта започва с дъска, състояща се от плочки с номера от 1 до N и една празна плочка, представена с цифрата 0. Целта е да се наредят плочките в съответствие с техните номера. Местенето се извършва, като на мястото на празната точка могат да се преместят плочки отгоре, отдолу, отляво и отдясно. 

На входа се подава число N - броя на плочките с номера, и след това се въвежда подредбата на дъската. С помощта на алгоритъма А* и евристиката "разстояние на Манхатън" да се изведе.

0. На първият ред дължината на пътя от началото до целевото състояние.
0. Съответните стъпки (на нов ред за всяка една), които се извършват за да се стигне до крайното състояние. Стъпките са left, right, up и down

**Примерен вход:**
```
8
1 2 3
4 5 6
0 7 8
```

**Примерен изход:**
```
2
left
left
```

## Валидация

Преди всичко ще използваме mergeSort който пресмята Inversion Count за да проверим дали даден пъзел е въобще решим.
Така си дефинираме

```python
check_puzzle(configuration)
```

която по входните данни, тоест наредбата ни казва дали даден пъзел е решим и неговото решение (крайно състояние).

In [10]:
import numpy

array = [1,2,3,4,5,6,0,7,8]

def mergeSort(A):
    if len(A)>1:
        x1, x2 = divide(A)
        x1, leftInv = mergeSort(x1)
        x2, rightInv = mergeSort(x2)
        merged, splitInv = merge(x1,x2)
        return merged, (leftInv + rightInv + splitInv)
    else:
        return A, 0

def divide(A):
    return A[:int(len(A)/2)], A[int(len(A)/2):]

def merge(x1,x2):
    x=[]
    x1Index=0
    x2Index=0
    invCount=0
    size1=len(x1)
    size2=len(x2)
    while ( x1Index < size1 and x2Index < size2):
        if x1[x1Index]<=x2[x2Index]:
            x.append(x1[x1Index])
            x1Index=x1Index+1
        else:
            x.append(x2[x2Index])
            x2Index=x2Index+1
            invCount += (size1-x1Index)

    for i in range(x1Index,size1):
         x.append(x1[i])

    for i in range(x2Index,size2):
         x.append(x2[i])

    return x, invCount

def check_puzzle(configuration):
    array, inv = mergeSort(configuration)
    
    array = numpy.roll(array, -1)
    
    return not inv % 2, array


solvable, solution = check_puzzle(array)

print('can be solved =', solvable, "and it's solution is =", solution)
    

can be solved = True and it's solution is = [1 2 3 4 5 6 7 8 0]


## Подготовка и манхатаново разстояние

За евристика на А* ще използваме както е по условие изискано - Манхатаново разстояние. Ще държим наредбата в масив и ще си смятаме всеки път координатите. 

In [34]:
import math

def coords_of(index, size):
    width = int(math.sqrt(size))
    
    x = int(index % width)
    y = int(index / width)
    
    return x, y
    
def value_of(x, y, array, size):
    width = int(math.sqrt(size))
    
    index = y * width + x 
    
    return array[index]

def manhattan_between(start, end, size):
    width = int(math.sqrt(size))
    
    start_coords = coords_of(start, size)
    end_coords = coords_of(end, size)
    
    return abs(start_coords[0] - end_coords[0]) + abs(start_coords[1] - end_coords[1])

## Да си имплементираме А*

In [107]:
import numpy
import queue
import math

EXAMPLE = [1,2,3,4,0,5,6,7,8]
EXAMPLE_2 = [1,2,3,4,5,6,7,8,0]

def mergeSort(A):
    if len(A)>1:
        x1, x2 = divide(A)
        x1, leftInv = mergeSort(x1)
        x2, rightInv = mergeSort(x2)
        merged, splitInv = merge(x1,x2)
        return merged, (leftInv + rightInv + splitInv)
    else:
        return A, 0

def divide(A):
    return A[:int(len(A)/2)], A[int(len(A)/2):]

def merge(x1,x2):
    x=[]
    x1Index=0
    x2Index=0
    invCount=0
    size1=len(x1)
    size2=len(x2)
    while ( x1Index < size1 and x2Index < size2):
        if x1[x1Index]<=x2[x2Index]:
            x.append(x1[x1Index])
            x1Index=x1Index+1
        else:
            x.append(x2[x2Index])
            x2Index=x2Index+1
            invCount += (size1-x1Index)

    for i in range(x1Index,size1):
         x.append(x1[i])

    for i in range(x2Index,size2):
         x.append(x2[i])

    return x,invCount

def check_puzzle(configuration):
    array, inv = mergeSort(configuration)
    
    array = numpy.roll(array, -1)
    
    return not inv % 2, array

def coords_of(index, size):
    width = int(math.sqrt(size))
    
    x = int(index % width)
    y = int(index / width)
    
    return x, y
    
def value_of(x, y, array, size):
    width = int(math.sqrt(size))
    
    index = y * width + x 
    
    return array[index]

def manhattan_between(start, end, size):
    width = int(math.sqrt(size))
    
    start_coords = coords_of(start, size)
    end_coords = coords_of(end, size)
    
    return abs(start_coords[0] - end_coords[0]) + abs(start_coords[1] - end_coords[1])

def a_star(start):
    
    def _available_moves(current):
        moves = []
        width = int(math.sqrt(len(start)))
        zero_index = current.index(0)
        zero_x, zero_y = coords_of(zero_index, len(start))
        
        if zero_x >= 0 and zero_x < width - 1:
            moves.append(_swap(current, zero_index, zero_index + 1))
        if zero_x > 0:
            moves.append(_swap(current, zero_index, zero_index - 1))
        if zero_y >= 0 and zero_y < width - 1:
            moves.append(_swap(current, zero_index, zero_index + width))
        if zero_y > 0:
            moves.append(_swap(current, zero_index, zero_index - width))

        return moves
    
    def _swap(current, first, second):
        current = list(current)
        current[first], current[second] = current[second], current[first]
        return ''.join(str(x) for x in current)
    
    solvable, solution = check_puzzle(start)
    
    if not solvable:
        print("Not solvable.")
    else:
        q = queue.PriorityQueue()
        q.put(start, 0)
        
        while not q.empty():
            current = q.get()
            current_str = ' '.join(str(x) for x in current)           
        
            if str(current) == str(list(solution)):
                break
        
            for next in _available_moves(current):
                print(next)
        
    
a_star(EXAMPLE)

123450678
123045678
123475608
103425678
